In [1]:
import torch
import torch.nn as nn
import sys
from config import input_size, proposalN, channels
from utils.read_dataset import read_dataset
from utils.auto_laod_resume import auto_load_resume
from networks.model import MainNet
from torchvision import transforms
import json
import imageio
import numpy as np
from PIL import Image

import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
DEVICE = torch.device("cpu")


In [3]:
root = './datasets/FGVC-aircraft'  # dataset path
# model path
pth_path = "./models/air_epoch146.pth"
num_classes = 100

batch_size = 10

In [4]:
_, testloader = read_dataset(input_size, batch_size, root, "Aircraft")

Loading Aircraft trainset
Loading Aircraft testset


In [5]:
model = MainNet(proposalN=proposalN, num_classes=num_classes, channels=channels)

/home/myoun/MMAL-Net/networks/resnet.py:224: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(pth_path)


In [6]:
model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()

In [7]:
if os.path.exists(pth_path):
    epoch = auto_load_resume(model, pth_path, status='test')
else:
    sys.exit('There is not a pth exist.')

/home/myoun/MMAL-Net/utils/auto_laod_resume.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location='cpu')


Load model from ./models/air_epoch146.pth
Resume from ./models/air_epoch146.pth


In [8]:
with open('id_label.json', 'r') as f:
    id_label_data = json.load(f)

id_label = {int(k): v for k, v in id_label_data.items()}

In [9]:
def preprocess_image(img):
    if len(img.shape) == 2:
        img = np.stack([img] * 3, 2)
    img = Image.fromarray(img, mode='RGB')
    img = transforms.Resize((448, 448), Image.BILINEAR)(img)
    img = transforms.ToTensor()(img)
    img = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(img)
    img = img.unsqueeze(0)
    return img

img = preprocess_image(imageio.imread("image.png"))

model.eval()

with torch.no_grad():
    output = model.forward(img, None, None, "test", "cpu")[-2:]
    pred = output[0].max(1, keepdim=True)[1]
    print(pred.numpy()[0][0], id_label[pred.numpy()[0][0]])

/tmp/ipykernel_93197/3254434968.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img = preprocess_image(imageio.imread("image.png"))


33 A380
